In [1]:
# For spotify api
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import cred

import scipy
import time
import json
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from scipy.signal import argrelextrema

In [8]:
# Getting manually classified playlists from my Spotify account
scope = "user-read-recently-played"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=cred.client_ID, 
                                               client_secret=cred.client_SECRET, 
                                               redirect_uri=cred.redirect_URI, 
                                               scope=scope))
results = sp.user_playlists(cred.user_ID)
playlist_id = results['items']
playlists = {item['id']: item['name'] for item in results['items']}

print("Playlists:", " / ".join(playlists.values()))

Playlists: Workout / Dislike / Non-Workout


In [3]:
# Getting track info and audio features from Spotify
dfs, tracks = [], []
for k, v in playlists.items():
    # Getting tracks
    combined, items, offset = [], [], 0
    while len(items) == 0 or len(items) == 100:  # NOTE more robust case check
        results = sp.playlist_items(k, offset=offset)
        items = results['items']
        combined = combined + items
        offset += 100
    print(v, len(combined))

    track_id, track_artist_name, track_title = [], [], []

    # Getting track info (artist, title, track_id)
    for item in combined:
        id = item['track']['id']
        artists = item['track']['album']['artists']
        artists_name = [artist['name'] for artist in artists][0]

        track_id.append(id)
        track_artist_name.append(artists_name)
        track_title.append(item['track']['name'])

    # Getting track audio features
    chunk_size = 100  # specifying chunk_size because the api accepts maximum 100 tracks
    track_id_sub = [track_id[i:i+chunk_size] for i in range(0, len(track_id), chunk_size)]
    track_id_audio_features = []
    for sublist in track_id_sub:
        track_id_audio_features = track_id_audio_features + sp.audio_features(sublist)

    dfs.append(pd.DataFrame(track_id_audio_features))
    tracks.append(pd.DataFrame({'id': track_id,
                                'artist': track_artist_name, 
                                'title': track_title}))


# Combining songs from different playlists into a single dataframe
workout, dislike, nonworkout = dfs[0], dfs[1], dfs[2]
dislike['result'], nonworkout['result'], workout['result'] = 0, 1, 2
song_df = pd.concat([workout, dislike, nonworkout])
song_df = song_df.drop_duplicates()
song_metainfo_df = pd.concat([tracks[0], tracks[1], tracks[2]]).drop_duplicates()
song_metainfo_df.to_csv('data/song_metainfo.csv', index=False)

print(f"All tracks are unique: {np.all(song_df['id'].value_counts() == 1)}")

Workout 766
Dislike 201
Non-Workout 411
All tracks are unique: True


In [4]:
song_df.sample(5)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,result
67,0.331,0.530,1,-5.805,1,0.0555,0.49000,0.000000,0.1270,0.217,78.611,audio_features,0kAllljYVZIuntRxyw9KOF,spotify:track:0kAllljYVZIuntRxyw9KOF,https://api.spotify.com/v1/tracks/0kAllljYVZIu...,https://api.spotify.com/v1/audio-analysis/0kAl...,197778,4,1
35,0.670,0.874,8,-5.221,1,0.0305,0.00231,0.000017,0.3000,0.789,130.041,audio_features,3dPtXHP0oXQ4HCWHsOA9js,spotify:track:3dPtXHP0oXQ4HCWHsOA9js,https://api.spotify.com/v1/tracks/3dPtXHP0oXQ4...,https://api.spotify.com/v1/audio-analysis/3dPt...,261013,4,2
483,0.670,0.739,10,-4.557,1,0.0323,0.00470,0.000005,0.1050,0.649,96.004,audio_features,0AcSCfVYcXgNgcrZdyZXOh,spotify:track:0AcSCfVYcXgNgcrZdyZXOh,https://api.spotify.com/v1/tracks/0AcSCfVYcXgN...,https://api.spotify.com/v1/audio-analysis/0AcS...,228120,4,2
765,0.901,0.637,0,-4.453,1,0.2540,0.08020,0.000000,0.0486,0.715,90.958,audio_features,5xTy9p0IXI8lRaU6iLGikC,spotify:track:5xTy9p0IXI8lRaU6iLGikC,https://api.spotify.com/v1/tracks/5xTy9p0IXI8l...,https://api.spotify.com/v1/audio-analysis/5xTy...,165333,4,2
154,0.626,0.703,11,-5.247,1,0.0306,0.02750,0.000002,0.0903,0.522,92.054,audio_features,4orHVYvdG5v4G4bmp2Lwdg,spotify:track:4orHVYvdG5v4G4bmp2Lwdg,https://api.spotify.com/v1/tracks/4orHVYvdG5v4...,https://api.spotify.com/v1/audio-analysis/4orH...,210373,4,1


## EDA

In [9]:
# For visualization purpose, 
# mapping each 2, 1, and 0 into Workout, Nonworkout, and Dislike
song_df['result'] = song_df['result'].replace({2: 'Workout',
                                               1: 'Nonworkout',
                                               0: 'Dislike'})

In [6]:
fig = px.bar(song_df['result'].value_counts(), title='Imbalanced target class distribution')
fig.update_layout(showlegend=False)
fig.update_traces(hovertemplate='Num of songs in %{x}: %{y}')
fig.show()
fig.write_html("assets/imbalanced_target_distribution.html")

In [7]:
fig = px.histogram(song_df, x='energy', color='result', opacity=0.6, 
                   barmode='overlay', title='Energy distribution by target class')
fig.update_layout(legend_title_text='')
fig.show()
fig.write_html("assets/energy_dist_by_target.html")

In [8]:
fig = px.scatter(song_df, x='valence', y='energy', color='result', title='Energy against valence by target class')
fig.show()
fig.write_html("assets/energy_against_valence_by_target.html")

## Collecting more data ([track audio analysis](https://developer.spotify.com/documentation/web-api/reference/get-audio-analysis)) for Feature Engineering

In [10]:
# Since retrieving track audio analysis takes time, I cached them in a json file

# Check for new songs that have been added to the playlists
with open("data/all_audio_analysis.json", 'r') as json_file:
    all_audio_analysis = json.load(json_file)

collected_tracks = all_audio_analysis.keys()
new_songs = song_df[~song_df['id'].apply(lambda id: id in collected_tracks)]

print(f"Number of new songs since last time: {len(new_songs)}")

Number of new songs since last time: 1


In [11]:
# Retrieve audio_analysis data from Spotify for the new songs and updates the local file

new_audio_analysis = {}
for id in new_songs['id']:
    try:
        audio_analysis = sp.audio_analysis(id)
        new_audio_analysis[id] = audio_analysis
        # To avoid exceeding rate limit
        time.sleep(5)
    except:
        print(f"{id} didn't work")

all_audio_analysis.update(new_audio_analysis)
with open("data/all_audio_analysis.json", "w") as outfile:
    json.dump(all_audio_analysis, outfile)

HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/5meVa5klVlJalupZTvv5XX with Params: {} returned 404 due to analysis not found


5meVa5klVlJalupZTvv5XX didn't work


In [9]:
# Load the latest version of audio_analysis file

with open("data/all_audio_analysis.json", 'r') as json_file:
    all_audio_analysis = json.load(json_file)

print(f"Number of songs in audio analysis: {len(all_audio_analysis)}")

Number of songs in audio analysis: 1337


## Feature Engineering

In [10]:
def get_descriptive_statistics(data):
    "Returns mean, std, median, skewness, kurtosis, and iqr"

    mean, std = np.mean(data), np.std(data)
    median = np.median(data)
    skewness = scipy.stats.skew(data)
    kurtosis = scipy.stats.kurtosis(data, fisher = True, bias = True)
    iqr = np.subtract(*np.percentile(data, [75, 25]))

    return [mean, std, median, skewness, kurtosis, iqr]


def count_local_max(arr):
    "Returns number of local max in arr"
    return len(argrelextrema(np.array(arr), np.greater)[0])


def pitches_analysis(audio_info):
    "Returns engineered features from pitches"

    pitches = [info['pitches'] for info in audio_info]
    pitches_df = pd.DataFrame(pitches)
    mean = pitches_df.mean().values

    max_pitch = pitches_df.apply(lambda row: row.idxmax(), axis=1)
    pitch_abs_diff = max_pitch.diff().iloc[1:].abs()
    pitch_abs_diff_mean = pitch_abs_diff.mean()
    pitch_abs_diff_std = pitch_abs_diff.std()

    pitches_sum = pitches_df.sum(axis=1)
    pitches_sum_mean = pitches_sum.mean()
    pitches_sum_std = pitches_sum.std()

    result = [list(mean), [pitch_abs_diff_mean, pitch_abs_diff_std, pitches_sum_mean, pitches_sum_std]]
    return [element for sublist in result if isinstance(sublist, list) for element in sublist]


def timbre_analysis(audio_info):
    "Returns timbre engineered features"

    result_df = pd.DataFrame()

    def process_timbre(category, df, result_df):
        stats = df.apply(get_descriptive_statistics).transpose()
        stats = stats.rename(columns={0:f'{category}_mean', 1:f'{category}_std', 
                                      2:f'{category}_median', 3:f'{category}_skewness',
                                      4:f'{category}_kurtosis', 5:f'{category}_iqr'})
        result_df = pd.concat([result_df, stats], axis=1)
        return result_df

    durations = [d['duration'] for d in audio_info]
    timbre = pd.DataFrame([d['timbre'] for d in audio_info])
    timbre_diff = timbre.diff()[1:]
    timbre_slope = timbre_diff.apply(lambda col: col / durations[:-1])
    result_df = process_timbre("default", timbre, result_df)
    result_df = process_timbre("diff", timbre_diff, result_df)
    result_df = process_timbre("slope", timbre_slope, result_df)

    return [element for sublist in result_df.to_numpy() for element in sublist]
    

def loudness_analysis(audio_info, results):
    "Returns loudness engineered features"

    variables = ['duration', 'loudness_max', 'loudness_start', 'loudness_max_time', 'loudness_end']
    for variable in variables:
        data = [info[variable] for info in audio_info]
        results[variable] = get_descriptive_statistics(data)

    loudness_start = [s['loudness_start'] for s in audio_info]
    loudness_max = [s['loudness_max'] for s in audio_info]
    loudness_gap = np.subtract(loudness_max, loudness_start)

    results['loudness_gap'] = [loudness_gap.mean(),
                               loudness_gap.sum(),
                               loudness_gap.std(),
                               len(loudness_gap),
                               count_local_max(loudness_gap)]
    return results


def min_max_timbre_analysis(audio_info):
    "Count the number of times each timbre is max/min in each segment"

    scaler = MinMaxScaler()
    # The first value represents the average loudness.
    # Thus, I didn't include it when evaluating relative order
    scaled_timbre = scaler.fit_transform(pd.DataFrame([s['timbre'][1:] for s in audio_info]))
    timbre_min = pd.Series([np.argmin(s) for s in scaled_timbre]).value_counts().sort_index()
    timbre_min = timbre_min.reindex(range(11), fill_value=0).values
    timbre_max = pd.Series([np.argmax(s) for s in scaled_timbre]).value_counts().sort_index()
    timbre_max = timbre_max.reindex(range(11), fill_value=0).values

    return np.ravel([timbre_min, timbre_max])


def segments_feature_engineering(audio_info):
    "Applying feature engineering to segments in audio analysis"

    results = {}
    results = loudness_analysis(audio_info, results)
    results['pitches'] = pitches_analysis(audio_info)
    results['timbre'] = timbre_analysis(audio_info)
    results['timbre_min_max'] = min_max_timbre_analysis(audio_info)

    return results

In [11]:
# Create a dataframe of engineered features

audio_analysis_dict, timbre_mean = {}, []
for audio_id, audio_analysis in all_audio_analysis.items():
    # segments feature engineering
    segments = audio_analysis['segments']
    desc_stat = segments_feature_engineering(segments)
    features = [element for sublist in desc_stat.values() for element in sublist]
    audio_analysis_dict[audio_id] = features

    # timbre mean for cosine similarity feature later
    timbre_mean.append(np.mean([s['timbre'] for s in segments], axis=0))

# Combining the two
fe_df = pd.DataFrame(audio_analysis_dict).transpose()
timbre_vector = pd.DataFrame(timbre_mean)
timbre_vector.columns = ['t1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12']
timbre_vector.index = fe_df.index
fe_df = fe_df.merge(timbre_vector, left_index=True, right_index=True)

fe_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12
1j795THd14Z0PmA3O18EwU,0.262939,0.141034,0.236735,3.589441,35.737605,0.165715,-3.643008,4.463240,-2.5845,-4.137120,...,19.097246,3.538538,33.994054,-18.063890,5.192796,0.070278,-8.601849,-1.257254,-15.284602,-6.575676
0dbp9YwlGEzXmxavRMgZ4T,0.285615,0.353402,0.226260,15.693871,305.722863,0.197865,-6.377804,4.814194,-5.7380,-2.819362,...,-10.835110,-12.803527,44.731583,-23.603955,-6.890832,-8.515410,-8.584296,2.839271,-13.565092,-6.618102
4t2FIqZJORKZGSKg30SShr,0.261434,0.150997,0.257370,5.302475,69.113209,0.155508,-8.195253,7.070832,-6.2295,-2.709352,...,-16.106525,1.452910,29.550585,-17.624864,4.026012,3.770445,-7.759859,0.134869,-15.092729,1.647532
6c6W25YoDGjTq3qSPOga5t,0.206486,0.238060,0.161970,25.423519,765.975913,0.133485,-8.906150,6.631479,-6.9670,-2.468119,...,-33.375089,5.339361,16.213991,-35.592518,3.503689,-8.963015,-11.297226,1.954241,-6.153882,1.620498
2QhURnm7mQDxBb5jWkbDug,0.301468,0.307902,0.246600,13.001216,226.876425,0.183405,-9.135564,5.833173,-7.9180,-3.527878,...,-10.900524,-12.185980,13.542297,-26.502399,-2.047738,-9.512079,-1.681319,0.473837,-10.338604,-4.943256


In [118]:
# combined track audio features with engineered features from track audio analysis

train = song_df.merge(fe_df, how='inner', left_on='id', right_index=True)
train = train.drop(columns=['uri', 'track_href', 'analysis_url', 'type'])
train = train.set_index('id')
train['result'] = train['result'].replace({"Workout": 2,
                                           "Nonworkout": 1,
                                           "Dislike": 0})

train.to_csv('data/train.csv')

In [10]:
train = pd.read_csv('data/train.csv')
train.head(5)

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12
0,0IGUXY4JbK18bu9oD4mPIm,0.755,0.870,1,-2.414,0,0.0936,0.00988,0.000000,0.0917,...,0.095460,-1.386235,17.568945,-17.985553,15.449831,3.269331,-15.614791,-2.964220,-15.069372,-0.568854
1,2iuZJX9X9P0GKaE93xcPjk,0.748,0.788,1,-7.055,1,0.0334,0.05910,0.000000,0.0863,...,-6.155959,-9.528637,24.578903,-18.166317,-11.814103,-0.269823,-2.291964,-4.685131,-9.972947,-3.879883
2,5qII2n90lVdPDcgXEEVHNy,0.608,0.768,0,-5.227,1,0.0475,0.09130,0.000000,0.0629,...,-0.892693,-2.473424,31.609930,-29.042121,2.096781,1.954543,-7.705413,0.504072,-9.815547,-2.803126
3,6ECp64rv50XVz93WvxXMGF,0.712,0.862,5,-4.612,0,0.0378,0.05250,0.000000,0.0930,...,16.693165,5.383770,31.743560,-21.515310,10.641415,3.251914,-7.824435,2.830839,-10.620220,-10.052625
4,6NnCWIWV740gP7DQ8kqdIE,0.726,0.910,4,-1.948,0,0.1790,0.04130,0.000008,0.4790,...,8.172904,5.951291,27.552778,-21.221918,19.413309,-0.647864,-11.209643,4.264785,-14.905512,0.921164


In [11]:
def cosine_similarity_features(df, timbre, vectors):
    # contains which class is the most similar
    cosine_similarity_result = []
    # contains raw similarity value for all classes
    cosine_similarity_raw_result = []

    # Calculate how much each song mean timbre values are similar to each class
    for _, row in df[timbre].iterrows():
        result = []
        for vector in vectors:
            result.append(cosine_similarity([row.values], [vector])[0][0])
        cosine_similarity_raw_result.append(result)
        cosine_similarity_result.append(np.argmax(result))

    # Combine results into a dataframe
    cosine_result = pd.concat([pd.DataFrame(cosine_similarity_raw_result), pd.DataFrame(cosine_similarity_result)], axis=1)
    cosine_result.columns = ['vec1', 'vec2', 'vec3', 'vec_argmax']
    cosine_result['vec_argmax'] = cosine_result['vec_argmax'].astype('category')
    cosine_result.index = df.index

    return pd.concat([df, cosine_result], axis=1)


def process(df, timbre, vectors, categorical_cols):
    "Add cosine similarity features and apply one hot encoding"
    df = cosine_similarity_features(df, timbre, vectors)
    df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
    return df


def split_to_train_test(df):
    # Split df into train/test
    X = df.drop(columns=['result'])
    y = df['result']
    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        y,
                                                        stratify=y,
                                                        shuffle=True,
                                                        test_size=0.2,
                                                        random_state=42)

    # Calculating mean timbre vector for each class in the training set
    vectors, timbre = [], [f't{i}' for i in range(1,13)]
    for i in range(3):
        vectors.append(X_train.loc[y_train[y_train==i].index][timbre].mean().values)
    X_train = process(X_train, timbre, vectors, ['key', 'mode', 'vec_argmax', 'time_signature'])
    X_test = process(X_test, timbre, vectors, ['key', 'mode', 'vec_argmax', 'time_signature'])
    X = process(X, timbre, vectors, ['key', 'mode', 'vec_argmax', 'time_signature'])

    return X_train, y_train, X_test, y_test, X, y

In [12]:
X_train, y_train, X_test, y_test, X, y = split_to_train_test(train)

In [13]:
X_train = X_train.set_index('id')
X_test = X_test.set_index('id')
X = X.set_index('id')

y_train.index = X_train.index
y_test.index = X_test.index
y.index = X.index

In [14]:
X_test.head(5)

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,key_8,key_9,key_10,key_11,mode_1,vec_argmax_1,vec_argmax_2,time_signature_3,time_signature_4,time_signature_5
id,,,,,,,,,,,,,,,,,,,,,
1Ame8XTX6QHY0l0ahqUhgv,0.519,0.595,-5.001,0.0283,0.00151,0.0,0.0762,0.237,129.786,247067,...,False,False,False,False,True,False,True,False,True,False
2As91j4G5GKRbx4In7ClyV,0.718,0.619,-6.788,0.0836,0.28600,0.0,0.0869,0.761,90.018,165792,...,False,False,True,False,False,False,False,False,True,False
0AkxBAeytxcvk6q2fMmFdh,0.490,0.762,-3.410,0.0367,0.00689,0.0,0.0812,0.386,109.761,187960,...,False,False,False,False,True,False,True,False,True,False
1ossvSyG2Z98nqBijLsDnF,0.719,0.508,-7.504,0.0477,0.33900,0.0,0.0927,0.582,95.015,195799,...,False,False,False,False,True,False,False,False,True,False
1zzxoZVylsna2BQB65Ppcb,0.761,0.899,-3.090,0.1830,0.01350,0.0,0.0719,0.673,95.027,217587,...,False,False,True,False,False,False,True,False,True,False


## PCA for dimension reduction

In [15]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)
X_scaled = scaler.fit_transform(X)

# Include components up to 95% variability
pca = PCA(n_components=0.95)
pca.fit(X_train_scaled)

X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)
X_pca = pca.transform(X_scaled)

print(f'Num of columns after PCA: {X_train_pca.shape[1]}')

Num of columns after PCA: 114


## LogisticRegression for classification

In [16]:
lr = LogisticRegression(solver='newton-cg', class_weight='balanced', max_iter=1000)
lr.fit(X_train_pca, y_train)
pred_test = lr.predict(X_test_pca)

# Calculating and printing the f1 score 
f1_test = f1_score(y_test, pred_test, average='weighted')
print('The f1 score for the testing data:', f1_test)

# Ploting the confusion matrix
confusion_matrix(y_test, pred_test)

The f1 score for the testing data: 0.6816972503451451


array([[15, 15, 10],
       [11, 66,  3],
       [16, 31, 98]])

In [17]:
# If I listen to 30 songs, on average, 3.5 songs are not from the Workout class
13/(13+98)*30

3.5135135135135136

In [18]:
pred = lr.predict(X_pca)
output = pd.Series(index=X.index, data=pred)
output_with_metainfo = pd.DataFrame(output).merge(song_metainfo_df, left_on='id', right_on='id')
output_with_metainfo = output_with_metainfo.rename(columns={0:'result'})
workout_output = output_with_metainfo[output_with_metainfo['result']==2].sample(40)
workout_output

,id,result,artist,title
463,4qRaZYiaOsSvWC7VgcxrI0,2,Fitz and The Tantrums,HandClap
1315,0KpAm8TtyHPZmxdPO7npYO,2,david hugo,we made it.
441,6zpiGeoHYRF5vtrboE3rAC,2,IU,Well… (ROCK VER.)
295,4QbdGSGyECE7IRtjolOkjO,2,(G)I-DLE,Peter Pan
723,5HJh0CWM5HBdbPA5pyHoil,2,Ethan Gander,APHRODITE
273,17SBD4aDKo9ZeahcQJ6j72,2,BTS,24/7=Heaven
573,2URMA0ap6SAI8wFmcY1yta,2,BLACKPINK,Really
667,6R1bxs1NBaUj1uDDHLKHXd,2,OH MY GIRL,Queen B
547,6LXMUR2rpxFBDo6nilS3yX,2,Imagine Dragons,Cool Out
532,4BYO8KU39k40yaRAwDgyUs,2,Trevor Wesley,Chivalry Is Dead
